# MMA_predictions
Using basic supervised learning to predict fight outcomes

The current dataset is restricted to the UFC

Have not fine tuned any of the models.
The Deep Learning models are WIP & not operational

The overall proecss of acquiring data and running the models needs to be automated.  

To run ...

## Step 1
Create "fight_card.xlsx" in the data directory with the following structure
weight_class      fighter1    fighter2
or populate the 'df_fight_card' DataFrame
This contains the bouts for which outcomes would get predicted

## Step 2
Execite the 'Load Dependencies' section

## Step 3
Execute the 'Load Datasets' section

## Step 4
Execute the 'Predict' section

Ed Bharucha 

# Load dependencies

In [1]:
import category_encoders as ce
import numpy as np
import pandas as pd
import pickle
import re
import requests
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import string
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb

warnings.filterwarnings("ignore")
# warnings.filterwarnings(action='once')

In [4]:
def scale(scaler, data):
  if (scaler == 'standard'):
    std_scaler = StandardScaler()
    return (std_scaler.fit_transform(data))
  elif (scaler == 'minmax'):
    minmax_scaler = MinMaxScaler(feature_range=(-1,1))
    return (minmax_scaler.fit_transform(data))

# Acquire & Prepare Fight Data

## Fighter stats

In [ ]:
first_names = []
last_names = []
nick_names = []
heights = []
weights = []
reaches = []
stances = []
wins = []
losses = []
draws = []
fighter_urls = []

In [ ]:
for c in list(string.ascii_lowercase):
    fighters_url = f'http://ufcstats.com/statistics/fighters?char={c}&page=all'
    
    page = requests.get(fighters_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    fighters_data = soup.find_all('tr')

    for fighter in fighters_data[2:]:
        fighter_data = fighter.find_all('td')
        first_names.append(fighter_data[0].get_text(strip=True))
        last_names.append(fighter_data[1].get_text(strip=True))
        nick_names.append(fighter_data[2].get_text(strip=True))
        heights.append(fighter_data[3].get_text(strip=True))
        weights.append(fighter_data[4].get_text(strip=True))
        reaches.append(fighter_data[5].get_text(strip=True))
        stances.append(fighter_data[6].get_text(strip=True))
        wins.append(fighter_data[7].get_text(strip=True))
        losses.append(fighter_data[8].get_text(strip=True))
        draws.append(fighter_data[9].get_text(strip=True))
        fighter_urls.append(fighter_data[0].find('a')['href'])

In [ ]:
fighter_data = pd.DataFrame(
    {
        'first_name' : first_names,
        'last_name' : last_names,
        'nick_name' : nick_names,
        'height' : heights,
        'weight' : weights,
        'reach' : reaches,
        'stance' : stances,
        'wins' : wins,
        'losses' : losses,
        'draws' : draws,
        'fighter_url' : fighter_urls,
    }
)

fighter_data.to_csv('data/fighter_data_base.csv')

In [ ]:
def height_cm(height):
    try:
        feet = height.split('"')[0].split("'")[0]
    except:
        feet = ''
    try:
        inches = height.split('"')[0].split("'")[1]
    except:
        inches = ''

    if not feet.strip().isdigit():
        feet = 0
    if not inches.strip().isdigit():
        inches = 0
    return (int(feet)*30.48 + int(inches)*2.54)

def reach_cm(reach):
    regnumber = re.compile(r'\d+(?:,\d*)?')
    rch = reach.split('"')[0]

    if regnumber.match(rch.strip()):
        inches = rch.split('.')[0]
    else:
        inches = 0
    
    return(int(inches)*2.54)

def weight_lbs(weight):
    lbs = weight.split(' ')[0]
    if not lbs.strip().isdigit():
        lbs = 0
    
    return(int(lbs))

In [ ]:
fighter_data['height'] = fighter_data['height'].apply(lambda x: height_cm(x))
fighter_data['reach'] = fighter_data['reach'].apply(lambda x: reach_cm(x))
fighter_data['weight'] = fighter_data['weight'].apply(lambda x: weight_lbs(x))

fighter_data.wins = pd.to_numeric(fighter_data.wins)
fighter_data.losses = pd.to_numeric(fighter_data.losses)
fighter_data.draws = pd.to_numeric(fighter_data.draws)

In [ ]:
SLpM = []
Str_Acc = []
SApM = []
Str_Dep = []
TD_Avg = []
TD_Acc = []
TD_Def = []
Sub_Avg = []

for url in tqdm(fighter_data.fighter_url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    career_stats = soup.find(class_='b-list__info-box b-list__info-box_style_middle-width js-guide clearfix').find_all('li')
    
    SLpM.append(career_stats[0].get_text(strip=True).split(':')[1])
    Str_Acc.append(career_stats[1].get_text(strip=True).split(':')[1])
    SApM.append(career_stats[2].get_text(strip=True).split(':')[1])
    Str_Dep.append(career_stats[3].get_text(strip=True).split(':')[1])
    TD_Avg.append(career_stats[5].get_text(strip=True).split(':')[1])
    TD_Acc.append(career_stats[6].get_text(strip=True).split(':')[1])
    TD_Def.append(career_stats[7].get_text(strip=True).split(':')[1])
    Sub_Avg.append(career_stats[8].get_text(strip=True).split(':')[1])


In [ ]:
fighter_data['full_name'] = fighter_data.first_name + " " + fighter_data.last_name

In [ ]:
# fighter_data.to_csv('../data/fighter_data.csv')
# with open ('../data/fighter_data.pkl', 'wb') as pklfile:
#     pickle.dump(fighter_data, pklfile)

with open ('../data/fighter_data.pkl', 'rb') as pklfile:
    fighter_data = pickle.load(pklfile)

## Fight results

In [ ]:
fight_data_url = 'http://ufcstats.com/statistics/events/completed?page=all'

page = requests.get(fight_data_url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
fight_list = soup.find('tbody').find_all('a')
fight_card_names = [fight.get_text(strip=True) for fight in fight_list]
fight_card_urls = [fight['href'] for fight in fight_list]

In [ ]:
fight_card = []
fight_card_url = []
fighter1 = []
fighter2 = []
weight_class = []
winner = []
method = []
round_ = []
i = 0

for url in tqdm(fight_card_urls[1:]):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    full_fight_card = soup.find('tbody')
    full_fight_card_fights = full_fight_card.find_all('tr')
    for fight in full_fight_card_fights:
        fight_card.append(fight_card_names[i+1])
        fight_card_url.append(url)
        fighter1.append(fight.find_all('td')[1].find_all('a')[0].get_text(strip=True))
        fighter2.append(fight.find_all('td')[1].find_all('a')[1].get_text(strip=True))
        weight_class.append(fight.find_all('td')[6].get_text(strip=True))
        winner.append(fighter1[-1])
        method.append(fight.find_all('td')[7].get_text(strip=True))
        round_.append(fight.find_all('td')[8].get_text(strip=True))
    i+=1

In [ ]:
fight_card_data = pd.DataFrame(
    {
        'fight_card' : fight_card,
        'fight_card_url' : fight_card_url,
        'fighter1' : fighter1,
        'fighter2' : fighter2,
        'weight_class' : weight_class,
        'winner' : winner,
        'method' : method,
        'round' : round_,
    }
)

fight_card_data.to_csv('../data/fight_card_data.csv')

In [ ]:
start_idx = 25
inc = 25
end_idx = start_idx + inc

fight_card_data_ = fight_card_data

while start_idx <= fight_card_data.shape[0]:
    fight_card_data_.loc[start_idx:end_idx,['fighter1', 'fighter2']] = fight_card_data_.loc[start_idx:end_idx,['fighter2', 'fighter1']].values
    start_idx = end_idx + 25
    end_idx = start_idx + inc

In [ ]:
# with open ('../data/fight_card_data.pkl', 'wb') as pklfile:
#     pickle.dump(fight_card_data, pklfile)

# with open ('../data/fight_card_data_.pkl', 'wb') as pklfile:
#     pickle.dump(fight_card_data_, pklfile)

# with open ('../data/fight_card_data.pkl', 'rb') as pklfile:
#     fight_card_data = pickle.load(pklfile)
    
with open ('../data/fight_card_data_.pkl', 'rb') as pklfile:
    fight_card_data_ = pickle.load(pklfile)    

## Consolidated dataset (fights + fighter stats.)

In [ ]:
fight_card_data_ = fight_card_data_.merge(fighter_data, left_on=['fighter1'], right_on=['full_name'], how='left').\
drop(columns=['first_name', 'last_name', 'nick_name', 'fighter_url'])

fight_card_data_.rename(columns={'height':'fighter1_height', 'weight':'fighter1_weight', 'reach':'fighter1_reach',\
                                 'stance':'fighter1_stance','wins':'fighter1_wins', 'losses':'fighter1_losses',\
                                 'draws':'fighter1_draws', 'SLpM':'fighter1_SLpM', 'Str_Acc':'fighter1_Str_Acc',\
                                 'SApM':'fighter1_SApM', 'Str_Dep':'fighter1_Str_Dep','TD_Avg':'fighter1_TD_Avg',\
                                 'TD_Acc':'fighter1_TD_Acc', 'TD_Def':'fighter1_TD_Def',\
                                 'Sub_Avg':'fighter1_Sub_Avg','full_name':'fighter1_full_name'}, inplace=True)

In [ ]:
fight_card_data_ = fight_card_data_.merge(fighter_data, left_on=['fighter2'], right_on=['full_name'], how='left').\
drop(columns=['first_name', 'last_name', 'nick_name', 'fighter_url'])

fight_card_data_.rename(columns={'height':'fighter2_height', 'weight':'fighter2_weight', 'reach':'fighter2_reach',\
                                 'stance':'fighter2_stance','wins':'fighter2_wins', 'losses':'fighter2_losses',\
                                 'draws':'fighter2_draws', 'SLpM':'fighter2_SLpM', 'Str_Acc':'fighter2_Str_Acc',\
                                 'SApM':'fighter2_SApM', 'Str_Dep':'fighter2_Str_Dep','TD_Avg':'fighter2_TD_Avg',\
                                 'TD_Acc':'fighter2_TD_Acc', 'TD_Def':'fighter2_TD_Def',\
                                 'Sub_Avg':'fighter2_Sub_Avg','full_name':'fighter2_full_name'}, inplace=True)

In [ ]:
# with open ('../data/fight_data_full.pkl', 'wb') as pklfile:
#     pickle.dump(fight_card_data_, pklfile)
    
with open ('../data/fight_data_full.pkl', 'rb') as pklfile:
    fight_data_full = pickle.load(pklfile)    

In [ ]:
fight_data_full.head()

## Finalize & Encode dataset

In [3]:
with open ('../data/fighter_data.pkl', 'rb') as pklfile:
    fighter_data = pickle.load(pklfile)

with open ('../data/fighter_data_en.pkl', 'rb') as pklfile:
    fighter_data_en = pickle.load(pklfile)    
    
with open ('../data/fight_card_data_.pkl', 'rb') as pklfile:
    fight_card_data_ = pickle.load(pklfile)   
    
with open ('../data/fight_data_full.pkl', 'rb') as pklfile:
    fight_data_full = pickle.load(pklfile) 

### Encode Fighter Data

In [11]:
print ([f'{col}=>{type(fighter_data[col][0])} ' for col in fighter_data.columns])

["first_name=><class 'str'> ", "last_name=><class 'str'> ", "nick_name=><class 'str'> ", "height=><class 'numpy.float64'> ", "weight=><class 'numpy.int64'> ", "reach=><class 'numpy.float64'> ", "stance=><class 'str'> ", "wins=><class 'numpy.int64'> ", "losses=><class 'numpy.int64'> ", "draws=><class 'numpy.int64'> ", "fighter_url=><class 'str'> ", "SLpM=><class 'str'> ", "Str_Acc=><class 'str'> ", "SApM=><class 'str'> ", "Str_Dep=><class 'str'> ", "TD_Avg=><class 'str'> ", "TD_Acc=><class 'str'> ", "TD_Def=><class 'str'> ", "Sub_Avg=><class 'str'> ", "full_name=><class 'str'> "]


In [ ]:
# Encode fighter data

import category_encoders as ce

enc = ce.HashingEncoder(cols = ['stance'])
fighter_data = enc.fit_transform(fighter_data)

fighter_data['SLpM'] = pd.to_numeric(fighter_data['SLpM'])

fighter_data['Str_Acc'] = fighter_data['Str_Acc'].str.replace('%', '')
fighter_data['Str_Acc'] = pd.to_numeric(fighter_data['Str_Acc'])

fighter_data['SApM'] = pd.to_numeric(fighter_data['SApM'])

fighter_data['Str_Dep'] = fighter_data['Str_Dep'].str.replace('%', '')
fighter_data['Str_Dep'] = pd.to_numeric(fighter_data['Str_Dep'])

fighter_data['TD_Avg'] = pd.to_numeric(fighter_data['TD_Avg'])

fighter_data['TD_Acc'] = fighter_data['TD_Acc'].str.replace('%', '')
fighter_data['TD_Acc'] = pd.to_numeric(fighter_data['TD_Acc'])

fighter_data['TD_Def'] = fighter_data['TD_Def'].str.replace('%', '')
fighter_data['TD_Def'] = pd.to_numeric(fighter_data['TD_Def'])

fighter_data['Sub_Avg'] = pd.to_numeric(fighter_data['Sub_Avg'])

# with open ('../data/fighter_data_en.pkl', 'wb') as pklfile:
#     pickle.dump(fighter_data, pklfile)

In [ ]:
fighter_full_name_unique = fighter_data.full_name.unique()

enc = ce.OrdinalEncoder(fighter_full_name_unique)
fighter_full_name_unique_enc = enc.fit_transform(fighter_full_name_unique)

In [ ]:
cols = ['fighter1', 'fighter2', 'weight_class','winner', \
        'fighter1_height', 'fighter1_weight','fighter1_reach', 'fighter1_stance',\
        'fighter1_wins', 'fighter1_losses', 'fighter1_draws', 'fighter1_SLpM',\
        'fighter1_Str_Acc', 'fighter1_SApM', 'fighter1_Str_Dep', 'fighter1_TD_Avg',\
        'fighter1_TD_Acc', 'fighter1_TD_Def', 'fighter1_Sub_Avg',\
        'fighter2_height', 'fighter2_weight', 'fighter2_reach', 'fighter2_stance',\
        'fighter2_wins', 'fighter2_losses', 'fighter2_draws', 'fighter2_SLpM',\
        'fighter2_Str_Acc', 'fighter2_SApM', 'fighter2_Str_Dep', 'fighter2_TD_Avg',\
        'fighter2_TD_Acc', 'fighter2_TD_Def', 'fighter2_Sub_Avg']

In [ ]:
fight_data_full_ = fight_data_full[cols]

In [ ]:
import category_encoders as ce

enc = ce.HashingEncoder(cols = ['weight_class', 'fighter1_stance', 'fighter2_stance'])
fight_data_full_ = enc.fit_transform(fight_data_full_)

### Prepare Relevant Data for Modeling

In [39]:
# Relevant Fighter Data

fighter_relevant_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'height', 'weight', 'reach',
       'wins', 'losses', 'draws', 'SLpM', 'Str_Acc', 'SApM',
       'Str_Dep', 'TD_Avg', 'TD_Acc', 'TD_Def', 'Sub_Avg', 'full_name']
fighter_data_relevant = fighter_data_en[fighter_relevant_cols]

In [40]:
fighter_data_relevant.columns

Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'height', 'weight', 'reach', 'wins', 'losses', 'draws', 'SLpM',
       'Str_Acc', 'SApM', 'Str_Dep', 'TD_Avg', 'TD_Acc', 'TD_Def', 'Sub_Avg',
       'full_name'],
      dtype='object')

In [41]:
# Relevant Fight Data

fight_data_relevant = fight_data_full[['fighter1', 'fighter2', 'weight_class', 'winner']]
fight_data_relevant.head()

,fighter1,fighter2,weight_class,winner
0,Stipe Miocic,Daniel Cormier,Heavyweight,Stipe Miocic
1,Nate Diaz,Anthony Pettis,Welterweight,Nate Diaz
2,Paulo Costa,Yoel Romero,Middleweight,Paulo Costa
3,Sodiq Yusuff,Gabriel Benitez,Featherweight,Sodiq Yusuff
4,Derek Brunson,Ian Heinisch,Middleweight,Derek Brunson


In [42]:
# Get data for fighter 1

fight_data_relevant = fight_data_relevant.merge(fighter_data_relevant, left_on=['fighter1'], right_on=['full_name'],\
                                                how='left').drop(columns=['full_name'])
fight_data_relevant.rename(columns={'col_0':'col_0_fighter1', 'col_1':'col_1_fighter1', 'col_2':'col_2_fighter1',\
                                    'col_3':'col_3_fighter1', 'col_4':'col_4_fighter1', 'col_5':'col_5_fighter1',\
                                    'col_6':'col_6_fighter1', 'col_7':'col_7_fighter1',\
                                    'height':'height_fighter1', 'weight':'weight_fighter1', 'reach':'reach_fighter1',\
                                    'wins':'wins_fighter1', 'losses':'losses_fighter1', 'draws':'draws_fighter1',\
                                    'SLpM':'SLpM_fighter1','Str_Acc':'Str_Acc_fighter1', 'SApM':'SApM_fighter1',\
                                    'Str_Dep':'Str_Dep_fighter1', 'TD_Avg':'TD_Avg_fighter1', 'TD_Acc':'TD_Acc_fighter1',\
                                    'TD_Def':'TD_Def_fighter1', 'Sub_Avg':'Sub_Avg_fighter1'}, inplace=True)

In [43]:
# Get data for fighter 2

fight_data_relevant = fight_data_relevant.merge(fighter_data_relevant, left_on=['fighter2'], right_on=['full_name'],\
                                                how='left').drop(columns=['full_name'])
fight_data_relevant.rename(columns={'col_0':'col_0_fighter2', 'col_1':'col_1_fighter2', 'col_2':'col_2_fighter2',\
                                    'col_3':'col_3_fighter2', 'col_4':'col_4_fighter2', 'col_5':'col_5_fighter2',\
                                    'col_6':'col_6_fighter2', 'col_7':'col_7_fighter2',\
                                    'height':'height_fighter2', 'weight':'weight_fighter2', 'reach':'reach_fighter2',\
                                    'wins':'wins_fighter2', 'losses':'losses_fighter2', 'draws':'draws_fighter2',\
                                    'SLpM':'SLpM_fighter2','Str_Acc':'Str_Acc_fighter2', 'SApM':'SApM_fighter2',\
                                    'Str_Dep':'Str_Dep_fighter2', 'TD_Avg':'TD_Avg_fighter2', 'TD_Acc':'TD_Acc_fighter2',\
                                    'TD_Def':'TD_Def_fighter2', 'Sub_Avg':'Sub_Avg_fighter2'}, inplace=True)

In [44]:
# Encode weight class

import category_encoders as ce

enc = ce.HashingEncoder(cols = ['weight_class'])
fight_data_relevant = enc.fit_transform(fight_data_relevant)
fight_data_relevant.rename(columns={'col_0':'col_0_weight_class', 'col_1':'col_1_weight_class', 'col_2':'col_2_weight_class',\
                                    'col_3':'col_3_weight_class', 'col_4':'col_4_weight_class', 'col_5':'col_5_weight_class',\
                                    'col_6':'col_6_weight_class', 'col_7':'col_7_weight_class'}, inplace=True)

In [54]:
# Encode winner

fight_data_relevant_en = fight_data_relevant.copy()
fight_data_relevant['winner_en'] = np.where(fight_data_relevant['winner']==fight_data_relevant['fighter1'], 0, 1)
fight_data_relevant.drop(columns=['fighter1', 'fighter2', 'winner'], inplace=True)


In [61]:
# with open ('../data/fight_data_relevant.pkl', 'wb') as pklfile:
#     pickle.dump(fight_data_relevant, pklfile)

### Misc Encoding

In [ ]:
fighter_full_name = pd.DataFrame()
fighter_full_name['full_name'] = fighter_full_name_unique
fighter_full_name['full_name_enc'] = fighter_full_name_unique_enc
fighter_full_name.head()

In [ ]:
# fight_data_full_ = fight_data_full_.merge(fighter_full_name, left_on=['fighter1'], right_on=['full_name'],\
#                                           how='left').drop(columns=['full_name', 'fighter1'])
# fight_data_full_.rename(columns={'full_name_enc':'fighter1_enc'}, inplace=True)

# fight_data_full_ = fight_data_full_.merge(fighter_full_name, left_on=['fighter2'], right_on=['full_name'],\
#                                           how='left').drop(columns=['full_name', 'fighter2'])
# fight_data_full_.rename(columns={'full_name_enc':'fighter2_enc'}, inplace=True)

# fight_data_full_ = fight_data_full_.merge(fighter_full_name, left_on=['winner'], right_on=['full_name'],\
#                                           how='left').drop(columns=['full_name', 'winner'])
# fight_data_full_.rename(columns={'full_name_enc':'winner_enc'}, inplace=True)

In [ ]:
fight_data_full_.tail()

In [ ]:
# fight_data_full_['round'] = pd.to_numeric(fight_data_full_['round'])

# fight_data_full_['fighter1_SLpM'] = pd.to_numeric(fight_data_full_['fighter1_SLpM'])

# fight_data_full_['fighter1_Str_Acc'] = fight_data_full_['fighter1_Str_Acc'].str.replace('%', '')
# fight_data_full_['fighter1_Str_Acc'] = pd.to_numeric(fight_data_full_['fighter1_Str_Acc'])

# fight_data_full_['fighter1_SApM'] = pd.to_numeric(fight_data_full_['fighter1_SApM'])

# fight_data_full_['fighter1_Str_Dep'] = fight_data_full_['fighter1_Str_Dep'].str.replace('%', '')
# fight_data_full_['fighter1_Str_Dep'] = pd.to_numeric(fight_data_full_['fighter1_Str_Dep'])

# fight_data_full_['fighter1_TD_Avg'] = pd.to_numeric(fight_data_full_['fighter1_TD_Avg'])

# fight_data_full_['fighter1_TD_Acc'] = fight_data_full_['fighter1_TD_Acc'].str.replace('%', '')
# fight_data_full_['fighter1_TD_Acc'] = pd.to_numeric(fight_data_full_['fighter1_TD_Acc'])

# fight_data_full_['fighter1_TD_Def'] = fight_data_full_['fighter1_TD_Def'].str.replace('%', '')
# fight_data_full_['fighter1_TD_Def'] = pd.to_numeric(fight_data_full_['fighter1_TD_Def'])

# fight_data_full_['fighter1_Sub_Avg'] = pd.to_numeric(fight_data_full_['fighter1_Sub_Avg'])

# fight_data_full_['fighter2_SLpM'] = pd.to_numeric(fight_data_full_['fighter2_SLpM'])
# fight_data_full_['fighter2_SApM'] = pd.to_numeric(fight_data_full_['fighter2_SApM'])
# fight_data_full_['fighter2_TD_Avg'] = pd.to_numeric(fight_data_full_['fighter2_TD_Avg'])
# fight_data_full_['fighter2_Sub_Avg'] = pd.to_numeric(fight_data_full_['fighter2_Sub_Avg'])

# fight_data_full_['fighter2_Str_Acc'] = fight_data_full_['fighter2_Str_Acc'].str.replace('%', '')
# fight_data_full_['fighter2_Str_Acc'] = pd.to_numeric(fight_data_full_['fighter2_Str_Acc'])

# fight_data_full_['fighter2_Str_Dep'] = fight_data_full_['fighter2_Str_Dep'].str.replace('%', '')
# fight_data_full_['fighter2_Str_Dep'] = pd.to_numeric(fight_data_full_['fighter2_Str_Dep'])

# fight_data_full_['fighter2_TD_Acc'] = fight_data_full_['fighter2_TD_Acc'].str.replace('%', '')
# fight_data_full_['fighter2_TD_Acc'] = pd.to_numeric(fight_data_full_['fighter2_TD_Acc'])

# fight_data_full_['fighter2_TD_Def'] = fight_data_full_['fighter2_TD_Def'].str.replace('%', '')
# fight_data_full_['fighter2_TD_Def'] = pd.to_numeric(fight_data_full_['fighter2_TD_Def'])



In [ ]:
for col in fight_data_full_.columns.tolist():
    print (f'{col} => {type(fight_data_full_.loc[0][col])}')

In [ ]:
# with open('../data/fighter_full_name.pkl', 'wb') as pklfile:
#     pickle.dump(fighter_full_name, pklfile)

# with open('../data/fight_data_full_en.pkl', 'wb') as pklfile:
#     pickle.dump(fight_data_full_, pklfile)

# Model

## Load Datasets

In [2]:
with open ('../data/fighter_data.pkl', 'rb') as pklfile:
    fighter_data = pickle.load(pklfile)

with open ('../data/fighter_data_en.pkl', 'rb') as pklfile:
    fighter_data_en = pickle.load(pklfile)

with open ('../data/fighter_data_relevant.pkl', 'rb') as pklfile:
    fighter_data_relevant = pickle.load(pklfile)    
    
with open ('../data/fighter_full_name.pkl', 'rb') as pklfile:
    fighter_full_name = pickle.load(pklfile)    

with open ('../data/fight_card_data_.pkl', 'rb') as pklfile:
    fight_card_data_ = pickle.load(pklfile)   
    
with open ('../data/fight_data_full.pkl', 'rb') as pklfile:
    fight_data_full = pickle.load(pklfile) 
    
with open ('../data/fight_data_full_en.pkl', 'rb') as pklfile:
    fight_data_full_en = pickle.load(pklfile)

with open ('../data/fight_data_relevant.pkl', 'rb') as pklfile:
    fight_data_relevant = pickle.load(pklfile)


## Evaluate

In [66]:
# X = scale('standard', fight_data_full_en.iloc[:,:-1])
# y = fight_data_full_en.iloc[:,-1]

X = scale('standard', fight_data_relevant.iloc[:,:-1])
y = fight_data_relevant.iloc[:,-1]

In [72]:
def pca (data):
  from sklearn.decomposition import PCA
#   pca = PCA(n_components=10)
  pca = PCA(.95)   # retain 95% of variance
  principalComponents_train = pca.fit_transform(data)
  pac_vars_train = [print (f'{pac_var*100:.2f}%\t', end="") for pac_var in pca.explained_variance_ratio_]
  print ()
  print (f'{np.sum(pca.explained_variance_ratio_)*100:.2f}%')
  return (principalComponents_train)

X_scaled = scale('standard', fight_data_relevant.iloc[:,:-1])
pc = pca(X_scaled)

11.96%	8.58%	5.86%	4.80%	4.68%	4.38%	4.16%	3.74%	3.51%	3.02%	2.79%	2.68%	2.60%	2.51%	2.42%	2.36%	2.31%	2.18%	2.01%	1.99%	1.87%	1.73%	1.63%	1.60%	1.43%	1.40%	1.37%	1.28%	1.24%	1.20%	1.04%	1.04%	
95.37%


### cross_val_score on ML Classifiers

In [73]:
# Random Forrest

# score_rf = cross_val_score(RandomForestClassifier(n_estimators=100), X, y, cv=3)
score_rf = cross_val_score(RandomForestClassifier(n_estimators=100), pc, y, cv=3)
np.mean(score_rf)

0.6488572686187761

In [75]:
# LightGBM

lgb_params = {
                 'boosting_type' : 'dart', 'objective' : 'binary','learning_rate' : 0.06,
                  'n_estimators' : 500,  'max_bin' : 100, 'n_jobs' : 2, 'num_leaves' : 50,
                 }


lgb_params = {
                 'n_estimators' : 500, 'boosting_type' : 'dart'
                 }

score_lgb = cross_val_score(lgb.LGBMClassifier(**lgb_params),X, y, cv=5,)
# score_lgb = cross_val_score(
#     lgb.LGBMClassifier(objective='binary', metric='binary_logloss', n_estimators=100, num_leaves=10,), 
#     pc, y, cv=5,)
np.mean(score_lgb)

0.7000424790790536

## Predict

In [11]:
# Load & prepare training dataset
X = scale('standard', fight_data_relevant.iloc[:,:-1])
y = fight_data_relevant.iloc[:,-1]

# Load & prepare dataset for prediction

# fighter_data[fighter_data['full_name'].str.contains('Poirier')].full_name, fight_data_full.weight_class.unique()
df_fight_card = pd.read_excel('../data/fight_card.xlsx')
# df_fight_card = pd.DataFrame(
#     {
#         'weight_class' : ["Women's Strawweight", "Welterweight", "Flyweight", "Women's Flyweight", "Middleweight", 
#                           "Lightweight"],
#         'fighter1' : ["Jessica Andrade", "Li Jingliang", "Kai Kara-France", "Wu Yanan", "Robert Whittaker",
#                       "Khabib Nurmagomedov",],
#         'fighter2' : ["Weili Zhang", "Elizeu Zaleski dos Santos", "Mark De La Rosa", "Mizuki Inoue", "Israel Adesanya",
#                       "Dustin Poirier"],
#     }
# )

# Encode weight class

import category_encoders as ce

enc = ce.HashingEncoder(cols = ['weight_class'])
X_test = enc.fit_transform(df_fight_card)
X_test.rename(columns={'col_0':'col_0_weight_class', 'col_1':'col_1_weight_class', 'col_2':'col_2_weight_class',\
                                    'col_3':'col_3_weight_class', 'col_4':'col_4_weight_class', 'col_5':'col_5_weight_class',\
                                    'col_6':'col_6_weight_class', 'col_7':'col_7_weight_class'}, inplace=True)

# Get data for fighter1

X_test = X_test.merge(fighter_data_relevant, left_on=['fighter1'], right_on=['full_name'],\
                                                how='left').drop(columns=['full_name', 'fighter1'])
X_test.rename(columns={'col_0':'col_0_fighter1', 'col_1':'col_1_fighter1', 'col_2':'col_2_fighter1',\
                                    'col_3':'col_3_fighter1', 'col_4':'col_4_fighter1', 'col_5':'col_5_fighter1',\
                                    'col_6':'col_6_fighter1', 'col_7':'col_7_fighter1',\
                                    'height':'height_fighter1', 'weight':'weight_fighter1', 'reach':'reach_fighter1',\
                                    'wins':'wins_fighter1', 'losses':'losses_fighter1', 'draws':'draws_fighter1',\
                                    'SLpM':'SLpM_fighter1','Str_Acc':'Str_Acc_fighter1', 'SApM':'SApM_fighter1',\
                                    'Str_Dep':'Str_Dep_fighter1', 'TD_Avg':'TD_Avg_fighter1', 'TD_Acc':'TD_Acc_fighter1',\
                                    'TD_Def':'TD_Def_fighter1', 'Sub_Avg':'Sub_Avg_fighter1'}, inplace=True)


# Get data for fighter 2

X_test = X_test.merge(fighter_data_relevant, left_on=['fighter2'], right_on=['full_name'],\
                                                how='left').drop(columns=['full_name', 'fighter2'])
X_test.rename(columns={'col_0':'col_0_fighter2', 'col_1':'col_1_fighter2', 'col_2':'col_2_fighter2',\
                                    'col_3':'col_3_fighter2', 'col_4':'col_4_fighter2', 'col_5':'col_5_fighter2',\
                                    'col_6':'col_6_fighter2', 'col_7':'col_7_fighter2',\
                                    'height':'height_fighter2', 'weight':'weight_fighter2', 'reach':'reach_fighter2',\
                                    'wins':'wins_fighter2', 'losses':'losses_fighter2', 'draws':'draws_fighter2',\
                                    'SLpM':'SLpM_fighter2','Str_Acc':'Str_Acc_fighter2', 'SApM':'SApM_fighter2',\
                                    'Str_Dep':'Str_Dep_fighter2', 'TD_Avg':'TD_Avg_fighter2', 'TD_Acc':'TD_Acc_fighter2',\
                                    'TD_Def':'TD_Def_fighter2', 'Sub_Avg':'Sub_Avg_fighter2'}, inplace=True)

X_test = scale('standard', X_test)

In [12]:
# LightGBM
import lightgbm as lgb

lgb_params = {
                 'n_estimators' : 500, 'boosting_type' : 'dart'
                 }

lgbm = lgb.LGBMClassifier(**lgb_params)
lgbm.fit(X, y)
# lgbm_score = lgbm.score(X, y)
# print (f'LightGBM score on test data = {lgbm_score*100:.2f}%')

y_pred_lgbm = lgbm.predict(X_test)
preds = []

for idx, val in enumerate(y_pred_lgbm):
    if (val == 0):
        preds.append(df_fight_card.fighter1[idx])
    else:
        preds.append(df_fight_card.fighter2[idx])

df_fight_card['Winner Prediction'] = preds

df_fight_card


,weight_class,fighter1,fighter2,Winner Prediction
0,Women's Strawweight,Jessica Andrade,Weili Zhang,Weili Zhang
1,Welterweight,Li Jingliang,Elizeu Zaleski dos Santos,Elizeu Zaleski dos Santos
2,Flyweight,Kai Kara-France,Mark De La Rosa,Kai Kara-France
3,Women's Flyweight,Wu Yanan,Mizuki Inoue,Wu Yanan
4,Middleweight,Robert Whittaker,Israel Adesanya,Israel Adesanya
5,Lightweight,Khabib Nurmagomedov,Dustin Poirier,Khabib Nurmagomedov


### Deep Learning

In [ ]:
import tensorflow as tf


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

def plot_hist(history):
  history_dict=history.history
  loss_values = history_dict['loss']
  val_loss_values=history_dict['val_loss']
  plt.figure(figsize=(10,6))
  plt.xlabel('Epochs')
  plt.plot(loss_values, color='Blue', linestyle='dashed', marker='o', label='Training Loss')
  plt.plot(val_loss_values,color='Red', label='Validation Loss')
  plt.legend()
  plt.show()

def model1(X, y):
  name = 'Model 1'

  input_shape = X[0].shape

  model = Sequential()
  model.add(Dense(256, input_shape=input_shape, activation='softmax'))
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(2, activation='linear'))
  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=[metrics.sparse_categorical_accuracy])

  # Runs model and assigns it to 'history'
  history = model.fit(X, y, epochs = 200, batch_size=32, validation_split = 0.2, verbose=0)

  y_pred = model.predict(X)
#   print(f'{name}: Accuracy the Train set is:\t{accuracy_score(y, y_pred):.3f}')
  
  # Plot Training & Validation Loss
  plot_hist(history)
  
  return(model)

In [ ]:
y_pred = model_1.predict(X)
y_pred[0]

In [ ]:
iters = 1

for iter in range(iters):
  model_1 = model1(X, y)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(y)
